# Apartment Sale Listings Research

You have access to data from the Yande service — a database of apartment sale listings in St. Petersburg and neighboring areas spanning several years. The goal is to learn how to determine the market value of real estate properties. Your task is to establish parameters. This will enable the creation of an automated system that can detect anomalies and fraudulent activities.

For each apartment for sale, two types of data are available. The first type is entered by the user, while the second type is automatically obtained based on cartographic data. For example, the distance to the city center, airport, nearest park, and water body.

## Studying data from a file

In [ ]:
import pandas as pd
from IPython.display import display

data = pd.read_csv('real_estate_data.csv')
display(data.head())

The data merged into a single line instead of being separated into columns. To split the columns, let's pass the `sep='\t'` parameter to the `read_csv()` method.

In [ ]:
data = pd.read_csv('real_estate_data.csv', sep='\t')
display(data.head())
data.info()

### Conclusion
The table consists of 22 columns and 23,699 rows. Each row in the table represents data about an apartment and its sale.

According to the documentation:
- `airports_nearest` — distance to the nearest airport in meters (m)
- `balcony` — number of balconies
- `ceiling_height` — ceiling height in meters (m)
- `cityCenters_nearest` — distance to the city center in meters (m)
- `days_exposition` — number of days the advertisement was active (from publication to withdrawal)
- `first_day_exposition` — publication date
- `floor` — floor
- `floors_total` — total number of floors in the building
- `is_apartment` — apartment (boolean type)
- `kitchen_area` — kitchen area in square meters (m²)
- `last_price` — price at the time of withdrawal from publication
- `living_area` — living area in square meters (m²)
- `locality_name` — locality name
- `open_plan` — open plan (boolean type)
- `parks_around3000` — number of parks within a 3 km radius
- `parks_nearest` — distance to the nearest park (m)
- `ponds_around3000` — number of ponds within a 3 km radius
- `ponds_nearest` — distance to the nearest pond (m)
- `rooms` — number of rooms
- `studio` — studio apartment (boolean type)
- `total_area` — total area of the apartment in square meters (m²)
- `total_images` — number of photos of the apartment in the listing

Only 8 columns have no missing values: `total_images`, `last_price`, `total_area`, `first_day_exposition`, `rooms`, `floor`, `studio`, `open_plan`. In columns such as `is_apartment`, `park_nearest`, `ponds_nearest`, there are the most missing values.

For half of the columns, the data type does not match reality. Columns like `floor_total`, `balcony`, `parks_around300`, `ponds_around3000`, `days_exposition` have a `float64` data type, although by nature, they should strictly be `int64`. Data in columns `airports_nearest`, `cityCenters_nearest`, `park_nearest`, and `ponds_nearest` may be floating-point numbers, but for convenience, it is advisable to change their type from `float64` to `int64`, especially since it will not affect the research results. The column `first_day_exposition` has an `object` type, which does not correspond to reality and should be `datatime64`. Similarly, the data type of the `is_apartment` column contradicts its nature, so it should be changed from `object` to `bool`.

There is a style violation in the column name `cityCenters_nearest`.

To proceed further, these data issues need to be addressed.

## Data Preprocessing

### Header Style
Let's rename the column `'cityCenters_nearest'` to `'city_centers_nearest'`:

In [ ]:
data.rename(columns={'cityCenters_nearest': 'city_centers_nearest'}, inplace=True)

First, let's calculate the number of missing values in the table.

In [ ]:
data.isna().sum()

First, let's fill in the gaps in the `floor_total` column. The presence of only a few gaps suggests they might be attributed to human error – someone may have simply overlooked filling in this data. We will fill in the gaps using the median value. If the floor level of an apartment is less than this median value, we will assume the total number of floors is equal to the apartment's floor level; otherwise, we will use the median value for the total floors.

In [ ]:
median_floors = data['floors_total'].median()

def total_floors(floor):
    return max(floor, median_floors)

mask = data['floors_total'].isna()
data.loc[mask, 'floors_total'] = data.loc[mask, 'floor'].apply(total_floors)

There are many missing values in the `ceiling_height` column, almost 40%. However, since the ceiling height is almost the same in all apartments, we will fill it with the median value.

In [ ]:
data['ceiling_height'].fillna(data['ceiling_height'].median(), inplace=True)

Next, let's fill in the gaps in the `kitchen_area` column, which occur in about 10% of cases. A pattern can be observed: for studios (`studio = True`), there is no kitchen area. On the one hand, this is indeed the case since in a studio, all rooms are combined into one. However, on the other hand, in reality, every apartment has a kitchen. Smaller apartments have smaller kitchens, and larger ones have larger ones. Let's examine the relationship between the kitchen area and the total apartment area.

In [ ]:
kitchen_total_ratio = data['kitchen_area'] / data['total_area']
kitchen_total_ratio

As we can see, the majority of kitchens occupy 0.15-0.2 of the total apartment area. Let's calculate the mean and median to see if our observations are confirmed.

In [ ]:
print(f'Mean ratio value: {kitchen_total_ratio.mean()}')
print(f'Median ratio: {kitchen_total_ratio.median()}')

Let's fill in the gaps in the `kitchen_area` column as `median ratio * total_area`.

In [ ]:
data['kitchen_area'].fillna(kitchen_total_ratio.median() * data['total_area'], inplace=True)

Using the same method, let's fill in the missing values in the `living_area` column:

In [ ]:
living_total_ratio_median = (data['living_area'] / data['total_area']).median()
data['living_area'].fillna(living_total_ratio_median * data['total_area'], inplace=True)

Most likely, the gaps in the `balcony` column have occurred because owners of apartments without balconies simply ignore this field in the form. Therefore, let's fill the `NaN` values with zeros.

In [ ]:
data['balcony'].fillna(0, inplace=True)

In the `is_apartment` column, there are 88% missing values. This is a very high percentage, so it is advisable to clarify information about these artifacts with the developer. Unfortunately, we do not have such an opportunity, so we will make assumptions ourselves. The first thing that comes to mind is that the `is_apartment`, `studio`, and `open_plan` columns are mutually exclusive. Let's check this:

In [ ]:
(data['is_apartment'] + data['studio'] + data['open_plan']).value_counts()

Indeed, if at least one of the three parameters is `True`, the other two are `False`. There are 72 such combinations in the table. However, there are also 2703 combinations where all three parameters are false (`is_apartment == False and studio == False and open_plan == False`). So, we can replace the missing values in `is_apartment` with `False` where at least one of the other two parameters (`studio`, `open_plan`) is `True`. However, we cannot replace the missing values with `True` where the other two columns are `False`. Therefore, it makes sense to replace all the missing values in this column with `False`.

In [ ]:
data['is_apartment'].fillna(False, inplace=True)

In the `locality_name` column, there are less than 0.5% missing values. Since we cannot find the names of the localities, we have two options: either delete the rows with missing values or replace the `NaN` values with a placeholder. Let's go with the first option:

In [ ]:
data.dropna(subset=['locality_name'], inplace=True)
data.reset_index(drop=True, inplace=True)

The `day_exposition` column informs us about how many days the ad was posted. However, if the ad is still active, `NaN` values are quite normal. Therefore, we won't do anything with the missing values in this column.

There are 20% missing values in the columns `airport_nearest`, `city_centers_nearest`, `parks_around3000`, `ponds_around3000`, and 60% missing values in the columns `parks_nearest`, `ponds_nearest`. If we take a closer look at the data, it's noticeable that the gaps in all six parameters occur simultaneously. Most likely, this coincidence occurred because the exact geolocation of the apartment was lost, and the program couldn't automatically calculate these data. However, we still have the name of the locality (`locality_name`) for these apartments. Therefore, we can fill the gaps with the median for each locality group.

Let's see what unique values are in the `locality_name` column:

In [ ]:
data['locality_name'].value_counts()

In the column, there are implicit duplicates caused by appending the settlement type name to the name of the settlement itself. For example, <i>посёлок Мурино, поселок Мурино, Мурино</i>. To address this, we will write a function that extracts only the capitalized word (i.e., the actual name) from the locality name strings.

In [ ]:
def get_locality_name(locality_name):
    return ' '.join([word for word in locality_name.split() if word[0].istitle()])

data['locality_name_unique'] = data['locality_name'].apply(get_locality_name)
data['locality_name_unique'].value_counts()

By removing duplicates, we have reduced the number of unique values by almost 20%. Now, we can fill in the gaps in the columns `airport_nearest`, `city_centers_nearest`, `parks_around3000`, `ponds_around3000`, `parks_nearest`, `ponds_nearest`.

In [ ]:
location_columns = ['airports_nearest', 'city_centers_nearest', 'parks_around3000', 'ponds_around3000', 'parks_nearest', 'ponds_nearest']

def get_value(row, column, medians):
    if pd.isna(row[column]):
        return medians.loc[row['locality_name_unique']]
    return row[column]

for col in location_columns:
    medians_data = data.groupby('locality_name_unique')[col].median()
    data[col] = data.apply(lambda row: get_value(row, col, medians_data), axis=1)

data.isna().sum()

Although some gaps still remain, we have managed to fill in over 600 gaps for each column. It doesn't make sense to fill the remaining gaps with specific data, as it may only distort the overall picture.

### Data Type Conversion

Let's take another look at the data types.

In [ ]:
data.info()

The columns `airports_nearest`, `city_centers_nearest`, `parks_around3000`, `parks_nearest`, `ponds_around3000`, `ponds_nearest` contain `NaN`, which is of type `float`, so we will not change the type in these columns. However, for the columns `last_price`, `total_area`, `floors_total`, `living_area`, `kitchen_area`, `balcony`, let's change the type from `float` to `int`. We will also change the date type to make it more convenient to work with.

In [ ]:
float_columns = ['last_price', 'total_area', 'floors_total', 'living_area', 'kitchen_area', 'balcony', 'rooms']

for col in float_columns:
    data[col] = data[col].astype('int')

data['first_day_exposition'] = pd.to_datetime(
    data['first_day_exposition'], format='%Y-%m-%dT%H:%M:%S'
)
data.info()

All the necessary data types have been changed.

### Handling Duplicates

Let's check the data for any duplicates.


In [ ]:
data.duplicated().sum()

There are no duplicates in the data.

## Calculations and Adding Results to the Table

Let's calculate the price per square meter:

In [ ]:
data['price_m2'] = (data['last_price'] / data['total_area']).round(2)

Let's find out and add to the table the day of the week, month, and year of the publication of the ad:

In [ ]:
data['weekday'] = data['first_day_exposition'].dt.day_name()
data['month'] = data['first_day_exposition'].dt.month_name()
data['year'] = data['first_day_exposition'].dt.year

Let's categorize the floor of the apartment - <i>первый, последний, другой</i>.

In [ ]:
def get_floor_category(row):
    if row['floor'] == 1:
        return 'первый'
    elif row['floor'] == row['floors_total']:
        return 'последний'
    return 'другой'

data['floor_category'] = data.apply(get_floor_category, axis=1)

Let's calculate the ratio of living area to total area and the ratio of kitchen area to total area.

In [ ]:
data['living_total_area_ratio'] = (data['living_area'] / data['total_area']).round(2)
data['kitchen_total_area_ratio'] = (data['kitchen_area'] / data['total_area']).round(2)

## Exploratory Data Analysis

Let's analyze the following characteristics: area, price, number of rooms, ceiling height, and the time it takes to sell the apartment by building histograms for each.

In [ ]:
data.hist('total_area')

From the histogram, it's evident that most apartments have an area below 100 m². However, there are apartments with an area reaching 900 m². It's challenging to imagine such apartments in real life, so most likely, these are outliers that distort the diagram. If we limit the plot to more realistic values (from 20 to 200 m²), we can analyze the data more effectively.

In [ ]:
data.hist('total_area', range=(20, 200), bins=20)
data['total_area'].describe()

The histogram has changed: there are more apartments (75%) with an area up to 70 m², and as the area increases, the number of apartments decreases exponentially. There are almost no apartments with an area of 200+ m².

Let's analyze the next column - `last_price`.

In [ ]:
data.hist('last_price')

It's challenging to pinpoint anything specific about this histogram. Most of the data lies within the range from 0 to 60,000,000. All other values seem more like outliers. Therefore, let's analyze the histogram limited to the numbers 0 and 0.6e8, and break it down into 50 bins.

In [ ]:
data.hist('last_price', bins=50, range=(0, 0.6e8))
data['last_price'].describe()

The histogram resembles a Poisson distribution. Most of the data falls within the range from 0 to 1e7, with the median at 0.5e7.

In [ ]:
data.hist('rooms')
data['rooms'].describe()

Most apartments have a number of rooms ranging from zero to three, while four- or five-room apartments are rare. The median and mean values are the same and equal to 2.

In [ ]:
data.hist('ceiling_height')

From this histogram, it's hard to make sense of anything. We need to set a `range` with more realistic ceiling heights.

In [ ]:
data.hist('ceiling_height', range=(2.35, 3.5))
data['ceiling_height'].describe()

Now we can analyze the plot. 50% of the data falls within the range from 2.6 to 2.7, which is a standard ceiling height in modern apartments. Of course, there are apartments with ceilings higher than 3m, possibly in historical or grand buildings.

In [ ]:
import matplotlib.pyplot as plt

def get_whiskers(dataset):
    q_1 = dataset.quantile(0.25)
    q_3 = dataset.quantile(0.75)
    iqr = q_3 - q_1
    left_whisker = q_1 - 1.5 * iqr
    right_whisker = q_3 + 1.5 * iqr
    return (
        max(dataset.min(), left_whisker),
        min(dataset.max(), right_whisker)
    )

columns = ['days_exposition', 'ceiling_height', 'rooms', 'total_area', 'last_price']

final_data = data
for col in columns:
    data.boxplot(column=col)
    plt.show()
    l_whisker, r_whisker = get_whiskers(data[col])
    final_data = final_data[(final_data[col] >= l_whisker) & (final_data[col] <= r_whisker)]

    data.plot(y=col, kind='hist', title=f'{col}')
    plt.show()

    final_data.plot(y=col, kind='hist', title=f'{col}_final')
    plt.show()
final_data.reset_index()
print(f'We removed {(1 - final_data.shape[0]/data.shape[0]):.2%} of the data.')

In [ ]:
data.hist('days_exposition', bins=50)
data['days_exposition'].describe()

The majority of apartments are sold within the first 100 days after the ad is published. The peak is between 30 and 60 days, after which it sharply declines. The standard deviation is large, and the mean is almost twice the median, indicating outliers that may have resulted from people forgetting to remove listings after selling their apartments.

### Removing Outliers

As we have already learned, our data is cluttered with rare outliers. To prevent them from distorting the overall picture, we should remove them.

Although we reduced the amount of data by a whopping 32%, we have eliminated all outliers: apartments with 18 rooms and ceilings of 100m. Now we can analyze the data and look for patterns.

### Which Factors Most Affect the Apartment Price?

Let's explore whether the price depends on the area, number of rooms, and distance from the city center. We can create pairwise scatter plots for price and area, price and number of rooms, price and distance from the center, and 13 more combinations. Additionally, we can calculate the correlation coefficient for each pair of columns.

In [ ]:
# let's create a function for future use

def corr_scatter_matrix(data, cols):
    fig, axes = plt.subplots(nrows=1, ncols=len(cols), figsize=(4*len(cols), 4))
    for i, col in enumerate(cols):
        data.plot(kind='scatter', x=col, y='last_price', ax=axes[i], sharey=True)
    plt.show()
    print('Correlation with last_price:',  data.corr().loc[cols, 'last_price'], sep='\n')

corr_scatter_matrix(final_data, ['total_area', 'rooms', 'city_centers_nearest'])

From the scatter plots, it is evident that the apartment price (`last_price`) is dependent on its area (`total_area`), meaning these two variables are correlated. Indeed, if we look at the correlation coefficient, it is 0.68, indicating a moderate correlation.

There is a weak correlation between parameters such as `last_price` and `rooms`. Studios and one-bedroom apartments are generally cheaper, but there isn't a significant price difference among apartments with 2 or more rooms.

A weak negative correlation exists between `last_price` and `city_centers_nearest`. From the scatter plot, we can see that apartments located far from the city center tend to be cheaper. However, it's also noticeable that apartments in the center or close to it can vary widely in price.

Let's check if the price depends on the day of the week, month, or year of the ad publication, as well as the floor category.

In [ ]:
# let's create a function for future use.
def scatter_plots(data, columns):
    for col in columns:
        plt.scatter(data['last_price'], data[col], alpha=0.1)
        plt.show()
        # let's also display the number of unique values and their mean for each column.
        display(data.pivot_table(index=col, values='last_price', aggfunc=['count', 'mean', 'median']))

scatter_plots(final_data, ['floor_category', 'weekday', 'month', 'year'])

Apartments on the first floor are the least expensive. Although the line representing apartments on the ***другом*** floor is denser than the line for apartments on the ***последнем*** floor, it doesn't necessarily mean these apartments are more expensive; it may be due to a larger amount of data. Moreover, the price range for apartments on the ***последнем*** floor corresponds to the price range for apartments on the ***другом*** floor.

The day of the week when the listing is published almost doesn't affect the apartment's price, except that ***понедельник*** has slightly fewer expensive apartments. It's also noticeable that on ***субботу*** and ***воскресенье***, people publish ads less actively.

The most advantageous month for buying an apartment is ***июле***, while the ***осенние месяцы*** might be less attractive for buyers. However, it's challenging to conclude that there is even a moderate correlation between the publication month and the price; it seems more like a weak or nonexistent relationship.

Looking at the scatter plot, one might infer that apartments were the cheapest in ***2015*** and ***2019***, while the most expensive in ***2017*** and ***2018***. However, for ***2015*** and ***2019***, we have less data, so their scatter plots look different. Nevertheless, if we consider the average apartment price over the last three years, we can observe a trend of increasing prices, which is logical considering inflation.

**Conclusion**
We have explored the relationship between the price and various characteristics of apartments. We can assert that there is a correlation between the apartment's area and the price, between the number of rooms and the price, as well as a moderate negative correlation between the price and the distance from the city center. The day and month of publication have no significant impact on the price, while there is a moderate connection with the year of publication.

Now let's select the top 10 localities with the highest number of listings. We will calculate the average price per square meter in these localities and identify the areas with the highest and lowest housing costs.

In [ ]:
(
    final_data.pivot_table(index='locality_name_unique', values='price_m2', aggfunc=['count', 'mean'])
    .sort_values(by=('count', 'price_m2'), ascending=False)
    [0:10]
    .sort_values(by=('mean', 'price_m2'), ascending=False)
)

In Санкт-Петербург, there is the largest number of listings, and it also has the highest price per square meter. In all other localities, the number of listings is at least 20 times less, and the average price per square meter differs by a couple of thousand. The lowest housing cost is in the city of Выборг.

We will investigate apartments in Санкт-Петербурге. To start, let's determine which area is considered the city center.

In [ ]:
data_spb = final_data.query('locality_name_unique == "Санкт-Петербург"').copy()
data_spb['city_centers_nearest_km'] = final_data['city_centers_nearest'] // 1000
(
    data_spb.pivot_table(index='city_centers_nearest_km', values='price_m2', aggfunc='mean')
    .plot(style='o-', grid=True, figsize=(14, 6))
)

From the plot, it is evident that the 8 km mark is a turning point. Beyond this distance, the price fluctuates but never exceeds the price at this point, indicating a decline.

Now let's focus on the segment of apartments in the city center and compare some average values with the overall averages for the entire city.

In [ ]:
data_spb_center = data_spb.query('city_centers_nearest_km < 8').copy()

columns = ['floors_total', 'ceiling_height', 'days_exposition', 'total_area', 'price_m2', 'rooms']

(
    pd.DataFrame(data_spb_center[columns].mean()).transpose()
    .append(
        pd.DataFrame(data_spb[columns].mean()).transpose()
    )
)

The city center is characterized by lower buildings compared to the entire city of St. Petersburg. However, the average ceiling height in these buildings is slightly higher. Apartments in the center tend to have larger total and living areas, as well as a higher number of rooms. Additionally, the price per square meter is higher in this area, but the average time for selling these apartments is longer.

If you have any specific questions or if there's anything else you would like to explore in the data, feel free to let me know!

Let's analyze the factors influencing the formation of housing prices and compare the results with the overall city data.

In [ ]:
corr_cols = ['total_area', 'rooms', 'ceiling_height', 'floor', 'city_centers_nearest', 'year']
print('Saint Petersburg:')
corr_scatter_matrix(data_spb, corr_cols)
print('\n')
print('The center of Saint Petersburg:')
corr_scatter_matrix(data_spb_center, corr_cols)

The highest correlation coefficient can be observed between the price and the total area of the apartment; it is slightly higher for the city center.

Since the number of rooms correlates with the total area, the price depends on the number of rooms accordingly.

In the city center, apartments located on higher floors are sold at a higher price; whereas, in the entire city dataset, the floor does not influence the price.

It is logical that for apartments in the center, the distance from the center has almost no effect on the price, but when analyzing the entire city, there is a small negative correlation between the price and the distance from the center.

Throughout the city, apartments with higher ceilings are sold at a higher price (correlation coefficient = 0.27), while for the center, the correlation coefficient is lower at 0.18.

Let's see how the prices of apartments in the center and the entire city change over the years:

In [ ]:
scatter_plots(data_spb, ['year'])
scatter_plots(data_spb_center, ['year'])

Interestingly, the cost of apartments does not change over time, and in the center, it even decreases. Now let's look at how the price per square meter changes.

In [ ]:
for data in [data_spb, data_spb_center]:
    data.plot(x='price_m2', y='year', kind='scatter', alpha=0.5)
    plt.show()
    display(data.pivot_table(index='year', values='price_m2', aggfunc=['mean', 'median']))

The price per square meter is still increasing. Now we can look at how the apartment's area changes. Let's create a pivot table:

In [ ]:
for data in [data_spb, data_spb_center]:
    display(
        data.pivot_table(
            index='year',
            values=['price_m2', 'total_area', 'last_price'],
            aggfunc=['mean']
        )
    )

With time, the average total area of sold apartments tends to decrease, while the price per square meter increases. This explains why the overall apartment price remains almost constant over the years or even decreases.

## Overall Conclusion

We have investigated real estate listings for apartments in St. Petersburg and its surroundings. We found that the following factors have the most significant impact on the apartment price:

- Total area of the apartment (correlation coefficient varies from 0.68 to 0.79 depending on the sample)
- Number of rooms (correlation coefficient ~0.4)
- Distance from the city center (correlation coefficient from -0.12 to -0.32, with an increase in distance from the center, the price decreases)
- Ceiling height (weak correlation in the range of 0.15–0.2)
- The floor has a slight impact, especially in the center of St. Petersburg (correlation coefficient 0.2)

The apartment price is not influenced by the day of the week, month, or year of publication. Although we discovered an interesting pattern that over the years, the total area of the sold apartments decreases, while the price per square meter increases.

In [ ]:
data = pd.Series([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
data.hist(bins=[0, 1, 4, 10])